<a href="https://colab.research.google.com/github/MDankloff/ClusterCompas/blob/main/COMPAS_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import DBSCAN
import warnings
warnings.filterwarnings("ignore")
import os
import random

# Load SHAP_ERROR_DATA

In [ ]:
Shap_error_data = pd.read_csv('/content/Shap_error_data.csv')
#Shap_error_data.info()
features = Shap_error_data.drop(['predicted_class', 'true_class', 'errors', 'TP', 'TN', 'FN', 'FP'], axis=1)

In [ ]:
print(Shap_error_data.isna().sum())
len(Shap_error_data.index)

age                              0
priors_count                     0
sex_Female                       0
sex_Male                         0
race_African-American            0
race_Asian                       0
race_Caucasian                   0
race_Hispanic                    0
race_Native American             0
race_Other                       0
Shap_age                         0
Shap_priors_count                0
Shap_sex_Female                  0
Shap_sex_Male                    0
Shap_race_African-American       0
Shap_race_Asian                  0
Shap_race_Caucasian              0
Shap_race_Hispanic               0
Shap_race_Native American        0
Shap_race_Other                  0
predicted_class               1508
true_class                    1508
errors                        1508
TP                            1508
TN                            1508
FN                            1508
FP                            1508
Error_Type                    1508
dtype: int64


5050

# Preprocessing: Initialize / scaling dataset

In [ ]:
def initialize_dataset (raw_data, with_errors = True, just_features = True, scale_features = True, with_classes = True):
  #input is the original dataset, whether errors are included, only features should be used, to scale the features, class labels are included
  #Initialisation of the dataset. Scales the features and errors, which can be included or exluded for clustering
  #it returns a scaled dataset with new columns "clusters" = 0 and "new_clusters" = -1, which is required for HBAC

  new_data = raw_data.copy(deep=True)

  if with_errors:
    scaling_factor = 0.8  # default scaling factor - there is a trade-off between scaling of weighing the errors to guide biased clusters while preventing too large and uninformative clusters
    new_data['scaled_TP'] = new_data['TP'] * scaling_factor
    new_data['scaled_TN'] = new_data['TN'] * scaling_factor
    new_data['scaled_FN'] = new_data['FN'] * scaling_factor
    new_data['scaled_FP'] = new_data['FP'] * scaling_factor

  if just_features:
    new_data = new_data.drop(['predicted_class', 'true_class', 'errors', 'TP', 'TN', 'FN', 'FP'], axis=1)

  if scale_features:
    to_scale = raw_data.drop(['predicted_class', 'true_class', 'errors', 'TP', 'TN', 'FN', 'FP'], axis=1).columns
    new_data[to_scale] = StandardScaler().fit_transform(features[to_scale])

  if with_classes:
    for col in ['predicted_class', 'true_class', 'errors', 'TP', 'TN', 'FN', 'FP']:
      new_data[col] = raw_data[col]


    new_data['clusters'] = 0
    new_data['new_clusters'] = -1
  return new_data

# PCA
needs to be separated still for TN/FP and TP/FN

In [ ]:
def pca_plot(data, title, alpha):

    #extract features for PCA and drop the other columns in other_features df
    pca_features = data.drop(['predicted_class', 'true_class', 'errors', 'TP', 'TN', 'FN', 'FP', 'Error_Type'], axis=1)
    other_features = data[['predicted_class', 'true_class', 'errors', 'TP', 'TN', 'FN', 'FP', 'Error_Type']]

    # Scale the PCA features before using PCA
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(pca_features)

    # Apply PCA with 2 components to scaled features and create a df for the resulting principal components
    pca_result = PCA(n_components=2).fit_transform(scaled_features)
    pca_df = pd.DataFrame(pca_result, index=pca_features.index)

    #create temporary dataset that contains both principal components and other features
    temp_dataset = pca_df.join(other_features, how='left')
    temp_dataset.rename(columns={0: 'Principal Component 1', 1: 'Principal Component 2'}, inplace=True)

    scatterplot = sns.scatterplot(data=temp_dataset,x ='Principal Component 1', y='Principal Component 2', alpha=alpha, hue="Error_Type", palette="tab10", style="Error_Type")

    scatterplot.set_title(title)
    scatterplot.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=1)

    explained_variance_ratio = PCA(n_components=2).fit(scaled_features).explained_variance_ratio_
    print(f"Explained Variance Ratio: PC1 = {explained_variance_ratio[0]:.2f}, PC2 = {explained_variance_ratio[1]:.2f}")

    plt.show()


## Clustering Functions

In [ ]:
#Calculate error metric

#Calculate bias

#Calculate absolute difference

#Calculate variance

#Get min splittable cluster size

##########################################

#Select a new cluster
#Returns cluster with highest absolute difference for the False positives (FP) and TN
#Cluster with highest absolute difference with 0.5 is selected as splitting cluster.
#Function requires a df.columns named 'clusters' and 'FP'
def select_new_cluster(data, FP, TN):
  highest_variance = -1
  cluster_number = 0

  for cluster_number in data['clusters'].unique():
    if cluster_number == -1:
      continue
    cluster_data = data[data['clusters'] == cluster_number]
    #FP and TN instead of error column
    mean_fp = cluster_data(['FP']).mean()
    mean_tn = cluster_data(['TN']).mean()
    mean_diff = abs(0.5 - mean_fp)

    if mean_diff > highest_variance:
      highest_variance = mean_diff
      selected_cluster = cluster_number

    return selected_cluster

#############################################

#Select a random cluster
#returns a random cluster from provided list of clusters that is not -1
def get_random_cluster(clusters):
  result = -1
  while (result == -1):
    result = random.randint(0, len(clusters.unique()))
  print('This is the random cluster we picked:', result)
  return result

#############################################

#Plot cluster
def plot_clusters(data):
  scatterplot = sns.scatterplot(data=data, x='1st', y='2nd', hue="clusters", size = 'errors', sizes=(100, 20), palette = "tab10")
  plt.show()


In [ ]:
def bias_with_error_rate(full_data, input_columns, error_column, max_iter=10, min_splittable_cluster_size=10, dbscan_max_iter=100):
#Cluster the data based on the FP or FN for error column and selects the best cluster iteratively.
  for i in range(1, max_iter):
